<a href="https://colab.research.google.com/github/Henry-0810/Artificial-Intelligence/blob/main/chinese_chess_search_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search Tree Algorithm Project
## End Game Steps of Chinese Chess (Xiang Qi)
Since full-game Chinese Chess is too complex, I plan to only focus on specific endgame scenarios.
1. King + Chariot vs. King (Basic but useful)
2. King + Cannon vs. King + Soldier (Intermediate)
3. King + Horse vs. King (More complex, requires mobility evaluation)

These scenarios are chosen because:
- The search tree remains manageable.
- The AI can calculate winning or drawing strategies.
- It demonstrates Minimax’s effectiveness.

---

**Some extra information about chinese chess:**
1. Chariot only can move up, down, left and right. It can capture any chess pieces, acts like a Rook in classic chess.
2. Cannon only can move up, down, left and right. A cannon must jump over a chess piece in its path to capture opponent's chess piece.
3. Soldier can only move one step forward, but once it moves pass the river, which is the mid line of the chess board, it can then move one step left, right and forward.
4. The Horse moves one point horizontally or vertically, and then one point diagonally. It cannot move in a direction where there is a piece blocking it along the path of movement.

---

**References:**
- [XiangQi Guide](https://www.xiangqi.com/how-to-play-xiangqi)

### **Implementation**

In [1]:
from copy import deepcopy

In [2]:
class CurrentBoard:
    def __init__(self, board_state=None):
        # 9x10 board represented as a list of lists
        if board_state:
            self.board = deepcopy(board_state)
        else:
            self.board = self.default_endgame_board()
        self.state = self.state_of_board()

    def default_endgame_board(self):
        # Simple endgame scenario: Red King vs Black King and Black Rook
        # Empty cells = '.', Red = 'R', Black = 'B'
        board = [['.' for _ in range(9)] for _ in range(10)]
        board[0][4] = 'BK'  # Black King
        board[0][0] = 'BR'  # Black Rook
        board[9][4] = 'RK'  # Red King
        return board

    def display(self):
        for row in self.board:
            print(" ".join(row))
        print("\n")

    def other(self, piece):
        return 'B' if piece == 'R' else 'R'

    def get_piece_owner(self, piece):
        return piece[0] if piece != '.' else None

    def state_of_board(self):
        # Simplified: check if either King is missing
        red_king = black_king = False
        for row in self.board:
            for cell in row:
                if cell == 'RK':
                    red_king = True
                if cell == 'BK':
                    black_king = True
        if not red_king:
            return 'B_WIN'
        if not black_king:
            return 'R_WIN'
        return 'U'

    def all_possible_moves(self, player):
        # Only implements basic king and rook logic
        directions = {
            'RK': [(-1, 0), (1, 0), (0, -1), (0, 1)],
            'BK': [(-1, 0), (1, 0), (0, -1), (0, 1)],
            'BR': [(-1, 0), (1, 0), (0, -1), (0, 1)]
        }
        moves = []
        for r in range(10):
            for c in range(9):
                piece = self.board[r][c]
                if piece != '.' and self.get_piece_owner(piece) == player:
                    for dr, dc in directions.get(piece, []):
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < 10 and 0 <= nc < 9:
                            target = self.board[nr][nc]
                            if target == '.' or self.get_piece_owner(target) != player:
                                new_board = deepcopy(self.board)
                                new_board[nr][nc] = piece
                                new_board[r][c] = '.'
                                moves.append(CurrentBoard(new_board))
        return moves

In [3]:
class SearchTreeNode:
    def __init__(self, board_instance, playing_as, ply=0):
        self.children = []
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as

        if self.current_board.state == 'U':
            self.generate_children()
        else:
            self.value = self.evaluate_terminal_state()
            self.value_is_assigned = True

    def evaluate_terminal_state(self):
        if self.current_board.state == 'D':
            return 0
        elif self.current_board.state == f'{self.move_for}_WIN':
            return 1
        else:
            return -1

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        child_values = [child.min_max_value() for child in self.children]
        if (self.ply_depth % 2) == 0:
            self.value = max(child_values)
        else:
            self.value = min(child_values)

        self.value_is_assigned = True
        return self.value

    def generate_children(self):
        for next_board in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(next_board, self.current_board.other(self.move_for), self.ply_depth + 1))


In [4]:
def play_xiangqi_endgame():
    cb = CurrentBoard()
    player = 'R'  # You play Red
    turn = 0

    while cb.state == 'U':
        print(f"Turn {turn}, {player}'s move")
        cb.display()

        if player == 'R':
            # Let human player select from all possible moves
            possible = cb.all_possible_moves(player)
            for idx, b in enumerate(possible):
                print(f"Move {idx}:")
                b.display()
            move = int(input("Choose move: "))
            cb = possible[move]
        else:
            tree = SearchTreeNode(cb, player)
            tree.min_max_value()
            best_move = max(tree.children, key=lambda x: x.value)
            cb = best_move.current_board

        player = cb.other(player)
        turn += 1

    print(f"Game Over! Winner: {cb.state}")
    cb.display()
